In [1]:
import numpy as np
import pandas as pd
import torch

In [2]:
import os

In [3]:
import pickle

In [4]:
import pylangacq as pla

In [5]:
import random

In [6]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
import nltk

Load Targets

In [9]:
os.getcwd()

'/Users/renee/Documents/CT_Fa18/Spec/Trace'

In [10]:
y_mem = torch.load('mem_mmse_dict.pt')

In [11]:
len(y_mem)

46

In [12]:
y_tech = torch.load('tech_mmse_dict.pt')

In [13]:
len(y_tech)

50

In [14]:
apc_datasheet = pd.read_excel('CogData_FU_82818.xlsx',header=None)

In [15]:
# column headers: 2-memory, 3-tech
apc_datasheet.iloc[0]

0                                             Record ID
1                                            Event Name
2     Write a short sketch about a memory from your ...
3     How does technology and social media impact th...
4                   Date of Neurobehavioral Status Exam
                            ...                        
92                         Cog State: One Back Accuracy
93                          Ravens Progressive Matrices
94                                Logical Memory Part B
95                         East Boston Immediate Recall
96                                            Complete?
Name: 0, Length: 97, dtype: object

In [31]:
apc_datasheet.iloc[list(y_mem.keys()),2]

6      When I was a little child I had the same exper...
29     When I was very young, living in the Astoria n...
51     I went to Paris when I was 16.  I was staying ...
56     My husband and I had always wanted to go to Pa...
78     I was born in Hungary. A significant memory is...
89     When I was 16 years old I traveled to Europe b...
100    I have great memories of growing up in huge ho...
107    When I was 8 or 9 years old, I was playing on ...
118    One of my fondest childhood memories is how mu...
154    One of my favorite memories from my childhood ...
184    My favorite pastime as a small child was re-en...
202    I remember going to the park with my dad to pl...
217    At the age of 14, I was the setter on a volley...
226    I have been able to recall - in detail -- memo...
242    I have a very clear memeory of last time I saw...
266    My parents lived in Ft. Myers, Florida, when I...
277    i visited Washington DC with my brother, my gr...
286    I remember finally going

In [33]:
apc_datasheet.iloc[list(y_tech.keys()),3]

6      I stay in touch with family and friends throug...
29     I'm a journalist who has covered the rise of t...
51     My husband and I are on our computers all day ...
56     I'm on the computer all the time, writing or r...
78     My use of social media is minimal. I only use ...
89     I check that on what my children are doing and...
100    Technology has helped me connect to family and...
107    I use a laptop and phone constantly for work a...
118    I do not use any social media sites at this ti...
154    Social media is great to stay in touch with wh...
184    Social Media is a bit of a challenge as I am a...
202    I use email daily for work and it usually cont...
217    Technology is woven into the fabric of our liv...
226    We dont engage on social media. Email exchange...
234    I'm writing correspondence via computer most o...
242    If by Technology we refer  to communication te...
266    Social media not much, since I don'tuse them, ...
277    I work in a business tha

Load Samples

---------------------------------------------------------------

Load Vocab Embeddings

In [16]:
glove_path = './PretrainedWordEmb/'

In [17]:
vocab_emb = pickle.load(open(f'{glove_path}/trace.vocab.glove.42B.300_words.pkl', 'rb'))

In [18]:
len(vocab_emb)

2125

In [19]:
trace_addb_posdict = torch.load('apc_pos_dict.pt')

In [44]:
# trace nltk pos: addb transcribed pos
trace_addb_posdict

{'CD': 'DET:NUM',
 'FW': 'FW',
 'EX': 'PRO:EXIST',
 'WP': 'PRO:INT',
 'NNS': 'N',
 'WRB': 'ADV',
 '(': '(',
 'RB': 'ADV',
 'PDT': 'DET:DEM',
 'VBG': 'PART',
 'CC': 'CONJ',
 "''": "''",
 ':': ':',
 'VBP': 'V',
 'IN': 'PREP',
 'TO': 'PREP',
 'MD': 'MOD',
 'PRP$': 'PRO:POSS',
 ')': ')',
 'PRP': 'PRO:PER',
 'JJ': 'ADJ',
 '.': '.',
 'JJR': 'ADJ',
 'VBD': 'V',
 'RBR': 'ADV',
 'RP': 'PART',
 ',': ',',
 'VB': 'V',
 'VBN': 'PART',
 'NN': 'N',
 'NNPS': 'N:PROP',
 'WDT': 'DET:POSS',
 'POS': 'DET:POSS',
 '$': '$',
 'NNP': 'N:PROP',
 'JJS': 'ADJ',
 'VBZ': 'V',
 'RBS': 'ADV',
 'DT': 'DET:ART'}

In [20]:
pos_dict = torch.load('./pos_dict.pt')

In [21]:
pos_dict

{'': 0,
 'CM': 1,
 'COMP': 2,
 'PRO:EXIST': 3,
 'IN#ADJ': 4,
 'PRE#PART': 5,
 '.': 6,
 'PRE#V': 7,
 'OVER#PART': 8,
 'UN#ADV': 9,
 'PRO:PER': 10,
 'ADV:TEM': 11,
 '+/.': 12,
 'V': 13,
 'PRO:INT': 14,
 'N': 15,
 'PRO:POSS': 16,
 'N:ADJ': 17,
 'DET:DEM': 18,
 'END': 19,
 'PRO:INDEF': 20,
 '+"/.': 21,
 'DET:POSS': 22,
 'OVER#V': 23,
 'DET:ART': 24,
 'INF': 25,
 'OVER#N:GERUND': 26,
 'META': 27,
 'ON': 28,
 'UP#V': 29,
 'MID#N': 30,
 'NEG': 31,
 'MOD': 32,
 '+...': 33,
 'PRO:SUB': 34,
 'UN#PART': 35,
 'PREP': 36,
 'DET:NUM': 37,
 'N:GERUND': 38,
 '+//?': 39,
 'UN#ADJ': 40,
 'COP': 41,
 'N:PT': 42,
 'AUX': 43,
 'ADV': 44,
 'OUT#PART': 45,
 '!': 46,
 'PRO:OBJ': 47,
 'ADJ': 48,
 'N:PROP': 49,
 'MINI#N': 50,
 '+/?': 51,
 '?': 52,
 'POST': 53,
 'N:LET': 54,
 'UN#N': 55,
 'UP#PART': 56,
 'COORD': 57,
 'QN': 58,
 'GRAND#N': 59,
 'CONJ': 60,
 '+..?': 61,
 'PART': 62,
 'BEG': 63,
 'PRO:REL': 64,
 'PRO:REFL': 65,
 'NEO': 66,
 'CO': 67,
 '+".': 68,
 'PRO:DEM': 69,
 '+//.': 70}

-------------------------------------------

Group minibatches of similar size 

-------------------------------------------

In [22]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [23]:
def get_embs(batchsize=1, mem_ix=[],tech_ix=[],pad=True):
    
    def list_tokenize(document):
        sentences = nltk.sent_tokenize(document)
        sentences = [nltk.word_tokenize(sent) for sent in sentences]
        
        return [token for sentence in sentences for token in sentence]
    
    def get_pos(document):
        #sentence segmentation
        sentences = nltk.sent_tokenize(document) 
        #tokenization
        sentences = [nltk.word_tokenize(sent) for sent in sentences]
        #pos tagging
        sentences = [nltk.pos_tag(sent) for sent in sentences] 

        pos_list = [p for sent in sentences for (t,p) in sent]
        return pos_list

    def num_vectorize(targets):
        vectors = []
        for i in targets:
            v = torch.zeros([30])
            # up until MMSE index...greater than index i 
            v[:i] = 1
            vectors.append(v)
        return torch.stack(vectors).double()
    
    def pad_minibatch(minib):
    
        batchsize = len(minib)

        seq_lens = [len(emb) for emb in minib]
        max_len = max(seq_lens)

        # input of shape (seq_len, batch, input_size): tensor containing the features of the input sequence
        # https://pytorch.org/docs/stable/generated/torch.nn.GRU.html
    #     seq_tensor = torch.zeros((batchsize, max_len, EMBEDDING_SIZE)).double()
        seq_tensor = torch.zeros((max_len, batchsize, EMBEDDING_SIZE)).double()


        for i, (seq,length) in enumerate( zip(minib,seq_lens) ):
            for wi,word in enumerate(seq):
    #             seq_tensor[i,wi] = word
                seq_tensor[wi,i] = word
        # mod to sort at the start?
        seq_tensor = nn.utils.rnn.pack_padded_sequence(seq_tensor,lengths=seq_lens,enforce_sorted=False)
        return seq_tensor

    
    embeddings = []
    targets = []


    for ix_list,data,targetdict in [(mem_ix,apc_datasheet[2],y_mem),(tech_ix,apc_datasheet[3],y_tech)]:
        
        for ix in ix_list:
            embedding = [(vocab_emb[token],torch.zeros(len(pos_dict),dtype=torch.float64),trace_addb_posdict[pos]) for (token,pos) in zip(list_tokenize(data.iloc[ix]), get_pos(data.iloc[ix]))]
            target = targetdict[ix]

            emb = []
            for tkn in embedding:
                if tkn[2] in pos_dict:
                    tkn[1][pos_dict[tkn[2]]] = 1
                    # parts of speech [')', '(', ',', '$', ':', 'FW', "''"] not in addb dataset --rm corresponding tokens (transcripts on which model was trained dont have foreign words, complete puctuation set)   
                    # see nltk.help.upenn_tagset() for parts of speech
                    # leaving in these tokens does not change accuracy but does increase average distance by 1
                    # removing all parts of speech reduces the accuracy and increases the average distance, indicating model relies on pos tag
                    emb.append(torch.cat((tkn[0],tkn[1])))

            embeddings.append(emb)

            # uncomment, rm emb.append(torch.cat((tkn[0],tkn[1]))) to include all tokens and unmapped pos as 0-vector
#             embeddings.append([torch.cat((tkn[0],tkn[1])) for tkn in embedding])
            targets.append(target)
    
    if pad: embeddings = pad_minibatch(embeddings)
# return target vals for accuracy 
    return embeddings, num_vectorize(targets), torch.tensor(targets)

In [25]:
# https://towardsdatascience.com/simple-trick-to-train-an-ordinal-regression-with-any-classifier-6911183d2a3c
# transform proba dist prediction to MMSE target
def out_to_score_proba(yhat_vector):
    # mod to 31 for ix 0-30
    proba_vector = torch.zeros([31])
    # no zero-score
    proba_vector[0] = 0. 
# P(MMSE=i) = P(MMSE>i-1) - P(MMSE>i)
    for i in range(1,len(yhat_vector)):
        proba_vector[i] = yhat_vector[i-1] - yhat_vector[i]
        
    proba_vector[-1] = yhat_vector[-1] 
    
#     print(proba_vector)
    return torch.argmax(proba_vector)


In [24]:
# not on y but on targets as mmse scores
def accuracy_distansum(yhat,y):
    return (torch.abs(y - yhat).sum())

In [26]:
# raw accurate prediction count
def accuracy(yhat_tensor,y_tensor):
#     return torch.stack([ya==yb for (ya,yb) in zip(yhat_tensor,y_tensor)]).sum()
    return ((yhat_tensor == y_tensor).sum())

------------------------------

MODEL 

2-layer rnn: gru units

In [27]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [30]:
HIDDEN_SIZE = 128
# EMBEDDING_SIZE = 371
EMBEDDING_SIZE = len(vocab_emb['I']) + len(pos_dict)

In [268]:
# max([len(emb) for emb in e])

546

In [35]:
# change if aadd in EOS, pos to 547
MAX_LENGTH = 546

In [31]:
VOCAB_SIZE = len(vocab_emb)

In [32]:
BATCH_SIZE = 32

In [36]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size=MAX_LENGTH, emb_size=EMBEDDING_SIZE, hidden_size=HIDDEN_SIZE):
        super(EncoderRNN, self).__init__()
        
        self.emb_size = emb_size
        self.hidden_size = hidden_size

        self.gru = nn.GRU(input_size=self.emb_size,hidden_size=self.hidden_size,num_layers=2).double()
#         # each output node Oi of our neural network
# uses a standard sigmoid function 1
# 1+e−zi
# , without including
# the outputs from other nodes, as shown in Figure 1. Output
# node Oi
# is used to estimate the probability oi
# that a data
# point belongs to category i independently, without subjecting
# to normalization as traditional neural networks do. Thus,
# for a data point x of category k, the target vector is
# (1, , 1, .., 1, 0, 0, 0), in which the first k elements is 1 and
# others 0
# http://orca.st.usm.edu/~zwang/files/rank.pdf A Neural Network Approach to Ordinal Regression
    
        self.fc = nn.Linear(self.hidden_size*2,30).double()
        
        self.activation = nn.Sigmoid()

    def forward(self, input, hidden):

        embedded = input.double()

        gru_out, _ = self.gru(embedded,hidden)
        
        hid_out = torch.cat((_[-2,:,:], _[-1,:,:]), dim = 1)

        out = self.fc(hid_out)
#         print(out.size())

        out - self.activation(out)

        return out, _
#         return torch.nn.utils.rnn.pad_packed_sequence(out), _

    def initHidden(self,batch_size=BATCH_SIZE):
        # h_0 of shape (num_layers * num_directions, batch, hidden_size): tensor containing the initial hidden state for each element in the batch. If the LSTM is bidirectional, num_directions should be 2
        return torch.zeros(2, batch_size, self.hidden_size, device=device).double()
        


In [33]:
# The
# cost function for a data point x can be relative entropy
# or square error between the target vector and the output
# vector
# loss = nn.BCELoss()
loss_func = nn.MSELoss()

In [38]:
encoder = EncoderRNN()

In [39]:
optimizer = torch.optim.Adam(encoder.parameters(), lr=1e-2)

In [41]:
encoder.load_state_dict(torch.load('./encoder_0.99accuracy_25epch.pt'))

<All keys matched successfully>

In [42]:
encoder.eval()

EncoderRNN(
  (gru): GRU(371, 128, num_layers=2)
  (fc): Linear(in_features=256, out_features=30, bias=True)
  (activation): Sigmoid()
)

In [43]:
x_trace,y_trace,targets_trace = get_embs(mem_ix=list(y_mem.keys()),tech_ix=list(y_tech.keys()))

In [44]:
len(y_mem) + len(y_tech)

96

In [45]:
_hidden = encoder.initHidden(batch_size=len(y_mem)+len(y_tech))

In [46]:
yhat, _hidden = encoder(x_trace, _hidden)
# Rm non-pos
estimates = torch.tensor([out_to_score_proba(hat) for hat in yhat])

In [51]:
# 9/96 accurate mmse prediction
accuracy(targets_trace,estimates)

tensor(9)

In [49]:
# avg mmse distance from target
# Any score of 24 or more (out of 30) indicates a normal cognition. Below this, scores can indicate severe (≤9 points), moderate (10–18 points) or mild (19–23 points) cognitive impairment.
# Crum RM, Anthony JC, Bassett SS, Folstein MF; Anthony; Bassett; Folstein (May 1993). "Population-based norms for the Mini-Mental Status Examination by age and educational level". JAMA. 269 (18): 2386–91. doi:10.1001/jama.1993.03500180078038. PMID 8479064.
# https://en.wikipedia.org/wiki/Mini%E2%80%93Mental_State_Examination#:~:text=Any%20score%20of%2024%20or,%E2%80%9323%20points)%20cognitive%20impairment.
accuracy_distansum(y=targets_trace,yhat=estimates)/len(estimates)

tensor(4)

In [192]:
estimates

tensor([29, 29, 28, 28, 23, 28, 29, 23, 16, 29, 28, 16,  3, 28, 28, 29, 30, 29,
        19, 28, 28, 29, 28, 29, 28, 23, 16, 24, 20, 29, 29, 13, 29, 28, 29, 28,
        23, 30, 20, 28, 26, 28, 28, 28, 23, 28, 23, 28,  5, 19, 19, 29, 19, 20,
        28, 19, 28, 28, 28, 20, 29, 30, 28, 28, 23, 19, 19, 28, 18, 17, 28, 28,
        25, 29, 28, 20, 28, 20, 19, 16, 19, 28, 23, 28, 28, 20, 28, 19, 28, 28,
        28, 23, 21, 29, 29, 28])

In [193]:
targets_trace

tensor([28, 30, 28, 30, 30, 30, 30, 29, 29, 29, 30, 30, 30, 29, 30, 30, 30, 30,
        28, 29, 30, 28, 30, 29, 30, 26, 30, 29, 29, 30, 30, 29, 30, 28, 30, 30,
        30, 30, 30, 30, 30, 30, 29, 30, 30, 29, 28, 30, 28, 30, 30, 30, 30, 29,
        29, 29, 30, 30, 30, 29, 29, 30, 30, 30, 30, 30, 28, 29, 30, 28, 28, 30,
        29, 30, 26, 30, 29, 29, 30, 30, 29, 30, 28, 30, 30, 30, 30, 30, 30, 30,
        30, 29, 30, 30, 30, 29])

In [130]:
9/96

0.09375

In [52]:
# memory samples -- accuracy 6/46 (.13)
accuracy(targets_trace[:46],estimates[:46])

tensor(6)

In [59]:
(accuracy_distansum(y=targets_trace[:46],yhat=estimates[:46]))/len(estimates[:46])

tensor(4)

In [54]:
# tech samples -- accuracy 3/50 (.06)
accuracy(targets_trace[46:],estimates[46:])

tensor(3)

In [60]:
(accuracy_distansum(y=targets_trace[46:],yhat=estimates[46:]))/len(targets_trace[46:])

tensor(5)

----------------------

In [126]:
# using all tokens, pos including nonmapped pos
estimates

tensor([29, 30, 28, 28, 23, 29, 29, 23, 16, 29, 28, 16,  3, 28, 28, 29, 28, 19,
        19, 28, 28, 29, 28, 29, 28, 23, 16, 24, 20, 29, 28, 13, 29, 28, 29, 28,
        23, 30, 20, 28, 26, 28, 28, 28, 23, 28, 23, 16,  5, 19, 13, 29, 19, 11,
        28, 19, 19,  3, 28, 28, 29, 30, 19, 28, 23, 19, 14, 28, 18, 17, 28, 28,
        16, 29, 28, 15, 28, 20,  3, 16, 19, 28, 23, 24, 28, 20, 28, 19, 28, 28,
        28, 23, 21, 29, 29, 28])

In [135]:
(torch.abs(targets_trace - estimates).sum())/len(estimates)

tensor(5)

In [162]:
# Without pos tags - ix 13 'I have been able to recall - in detail -- memories from my first year.  This was confirmed by my Mother.' jumps from 3 to 19 
# ...but pos is imp...tweak pos output?
estimates

tensor([29, 29, 20, 28, 23, 29, 29, 23, 16, 29, 28, 16, 19, 28, 28, 29, 28, 19,
        19, 25, 28, 29, 17, 29, 28, 23, 16, 24, 20, 29, 17, 29, 29, 28, 29, 28,
        23, 30, 29, 28, 26, 28, 28, 19, 14, 28, 23, 16,  5, 25, 13, 29, 19, 19,
        25, 19, 19,  3, 28, 20, 25, 30, 19, 29, 26, 19, 14, 28, 18, 17, 20, 28,
        16, 29, 28, 15, 28, 20,  5, 16, 19, 20, 23, 24, 28, 20, 13, 19, 16, 28,
        28, 23, 28, 19, 29, 28])

In [163]:
# distance without pos tags (all pos replaced by 0-vector)
(torch.abs(targets_trace - estimates).sum())/len(estimates)

tensor(6)

In [132]:
targets_trace

tensor([28, 30, 28, 30, 30, 30, 30, 29, 29, 29, 30, 30, 30, 29, 30, 30, 30, 30,
        28, 29, 30, 28, 30, 29, 30, 26, 30, 29, 29, 30, 30, 29, 30, 28, 30, 30,
        30, 30, 30, 30, 30, 30, 29, 30, 30, 29, 28, 30, 28, 30, 30, 30, 30, 29,
        29, 29, 30, 30, 30, 29, 29, 30, 30, 30, 30, 30, 28, 29, 30, 28, 28, 30,
        29, 30, 26, 30, 29, 29, 30, 30, 29, 30, 28, 30, 30, 30, 30, 30, 30, 30,
        30, 29, 30, 30, 30, 29])

In [62]:
# Export for heatmap visualization
pd.Series(targets_trace).to_csv('targets_trace.csv')

pd.Series(estimates).to_csv('estimates_trace.csv')

In [255]:
# compare performance with eval on validation addb set -- .99 accuracy 
encoder.eval()

x_valid,y_valid,targets_valid = get_minibatch(ix=valid_ix)

# x_valid = pad_minibatch(x_valid)

# y_valid = torch.stack(y_valid).double()

_hidden = encoder.initHidden(batch_size=len(valid_ix))

yhat_valid, _hidden = encoder(x_valid, _hidden)
estimates = torch.tensor([out_to_score_proba(hat) for hat in yhat_valid])


torch.Size([158, 30])


In [264]:
targets_valid

tensor([30, 18, 29, 28, 26, 27, 23, 30, 19, 17, 20, 17, 25, 30, 23, 29, 27, 30,
        19, 16, 30,  7, 28, 28, 28, 28, 25, 12, 30, 18, 30, 25, 15, 18, 14, 26,
        27, 30, 20, 17, 30, 26, 19, 27, 17, 20, 20, 29, 27, 29, 30, 30, 30, 20,
        22, 29, 19, 30, 30, 30, 30, 22, 17, 27, 27, 30, 30, 23, 30, 30, 19, 25,
        30, 19, 30, 30, 30,  8, 30, 22, 30, 30, 28, 16, 19, 22, 24, 28, 16, 30,
        30, 19, 17, 27, 19, 30, 30, 21, 28, 19, 29, 19, 12, 28, 20, 23, 13, 20,
        23, 27, 23, 19, 20, 19, 19, 19, 27, 16, 10, 20, 17, 24, 24, 23, 16, 20,
        18, 17, 19, 28, 29, 20, 27, 20, 23, 16, 20, 23, 27, 22, 19, 20, 20, 19,
        19, 28, 28, 28, 26, 19, 24, 22, 20, 18, 23, 27, 25, 25])

In [312]:
estimates

tensor([30, 18, 29, 30, 26, 27, 23, 30, 19, 17, 20, 17, 25, 30, 23, 29, 27, 30,
        19, 16, 30,  7, 28, 28, 28, 28, 25, 12, 30, 18, 30, 25, 15, 18, 14, 26,
        27, 30, 20, 17, 30, 26, 19, 27, 17, 20, 20, 29, 27, 29, 30, 30, 30, 20,
        22, 29, 19, 30, 30, 30, 30, 22, 17, 27, 27, 30, 30, 23, 30, 30, 19, 25,
        30, 19, 30, 30, 30,  8, 30, 22, 30, 30, 28, 16, 19, 22, 24, 28, 16, 30,
        30, 19, 17, 27, 19, 30, 30, 21, 28, 19, 29, 19, 12, 28, 20, 23, 13, 20,
        23, 27, 23, 19, 20, 19, 19, 19, 27, 16, 10, 20, 17, 24, 24, 23, 16, 20,
        18, 17, 19, 28, 29, 20, 27, 20, 23, 16, 20, 23, 27, 22, 19, 20, 20, 19,
        19, 28, 28, 28, 26, 19, 24, 22, 20, 18, 23, 27, 25, 25])

In [257]:
accuracy(targets_valid,estimates)

tensor(157)

In [258]:
157/158

0.9936708860759493

In [259]:
np.where(targets_valid != estimates)

(array([3]),)